In [868]:
import os
import zipfile
import pandas as pd
import numpy as np

# import plotly as plt

# Display all of the columns
pd.set_option('display.max_columns', None)

# File I/O

In [869]:
WRITE_FILES = True

if not WRITE_FILES:
    print("WARNING: NOT WRITING OUTPUT FILES!")
else:
    print("Outputs will be written to CSV files")

Outputs will be written to CSV files


In [870]:
def write_csv(df, output_path, filename_without_extension, zip=False):
    if WRITE_FILES:        
        if zip:
            compression = { 'method': 'zip',
                            'archive_name': filename_without_extension + '.csv' }
            write_extension = '.zip'
            print(f'Zipping "{filename_without_extension}.csv" into "{filename_without_extension}.zip"')
        else:
            compression = None
            write_extension = '.csv'
        print(f'Writing to "{output_path}{filename_without_extension}{write_extension}"')
        df.to_csv(os.path.join(output_path, filename_without_extension + write_extension), index=False, compression=compression)
    else:
        print("WARNING: NOT WRITING OUTPUT FILES!")


In [871]:
# function to open zipped file and read into df
def unzip_to_df(zip_filepath, file_inside_zip, dtype = {}, parse_dates = [], **read_csv_kwargs):
    try:
        with zipfile.ZipFile(zip_filepath, 'r') as z:
            with z.open(file_inside_zip) as f:
                df = pd.read_csv(f, dtype=dtype, parse_dates=parse_dates, **read_csv_kwargs)
        return df
    except Exception as e:
        print(f'Error occured: {e}')
        return None

## Paths and File Listings

In [872]:
root_folder = '../../..'
data_folder = root_folder + '/Resources'
root_cleaned_folder = root_folder + '/Preprocessing/Output'

delays_folder = data_folder + '/Delays/'
aircraft_folder = data_folder + '/Aircraft/'
airport_folder = data_folder + '/Airports/'
weather_folder = root_cleaned_folder + '/Weather/'
aircraft_zip_file = 'Aircraft.zip'
aircraft_reference_file = 'ACFTREF.csv'
aircraft_master_file = 'MASTER.csv'
airlines_path = airport_folder + 'airlines.csv'
airports_path = airport_folder + 'airports.csv'

print('Delays files:')
delays_files = [f"{filename}" for filename in os.listdir(delays_folder) if os.path.isfile((os.path.join(delays_folder, filename)))]
[print(filename) for filename in delays_files];

print('\nWeather files:')
weather_files = [f"{filename}" for filename in os.listdir(weather_folder) if os.path.isfile((os.path.join(weather_folder, filename)))]
[print(filename) for filename in weather_files];

Delays files:
AA-American_Airlines.csv
AS-Alaska_Airlines.csv
B6-JetBlue_Airways.csv
DH-Independence_Air.csv
DL-Delta_Airlines.csv
F9-Frontier_Airlines.csv
G4-Allegiant.csv
HA-Hawaiian Airlines.csv
MQ-American_Eagle_Airlines.csv
NK-Spirit_Airlines.csv
OO-SkyWest_Airlines.csv
QX-Horizon_Air.csv
UA-United_Airlines.csv
WN-Southwest_Airlines.csv

Weather files:
ABQ_2018-2025_cleaned.zip
ACV_2018-2025_cleaned.zip
ANC_2018-2025_cleaned.zip
ASE_2018-2025_cleaned.zip
ATL_2018-2025_cleaned.zip
AUS_2018-2025_cleaned.zip
BDL_2018-2025_cleaned.zip
BHM_2018-2025_cleaned.zip
BIH_2018-2025_cleaned.zip
BIL_2018-2025_cleaned.zip
BLI_2018-2025_cleaned.zip
BNA_2018-2025_cleaned.zip
BOI_2018-2025_cleaned.zip
BOS_2018-2025_cleaned.zip
BTR_2018-2025_cleaned.zip
BUF_2018-2025_cleaned.zip
BUR_2018-2025_cleaned.zip
BWI_2018-2025_cleaned.zip
BZN_2018-2025_cleaned.zip
CHS_2018-2025_cleaned.zip
CID_2018-2025_cleaned.zip
CLE_2018-2025_cleaned.zip
CLT_2018-2025_cleaned.zip
CMH_2018-2025_cleaned.zip
COS_2018-2025_cl

## Read Weather Data

In [873]:
# read weather data
origin_airport = 'LAX'
weather_destination_list = []
weather_origin_list = []

for filename in weather_files:    
    print(f'Processing file : {filename}')
    filename_csv = filename.replace('.zip', '.csv')
    dtype = {
        'cloud_layer_1_code': 'Int64',
        'cloud_layer_2_code': 'Int64',
        'cloud_layer_3_code': 'Int64'
    }
    df = unzip_to_df(weather_folder + filename, filename_csv, dtype=dtype, parse_dates=['DateTime'])
    airport = filename_csv[0:3]
    df['Airport'] = airport

    if airport == origin_airport:
        weather_origin_list.append(df)
    else:
        weather_destination_list.append(df)

weather_df = pd.concat(weather_origin_list)
weather_df = weather_df.reset_index(drop=True)

print(weather_df.dtypes)
display(weather_df.head())

weather_destination_df = pd.concat(weather_destination_list)
weather_destination_df = weather_destination_df.rename({'DateTime': 'Destination Weather DateTime'}, axis=1)
weather_destination_df = weather_destination_df.reset_index(drop=True)

print(weather_destination_df.dtypes)
display(weather_destination_df.head())

Processing file : ABQ_2018-2025_cleaned.zip
Processing file : ACV_2018-2025_cleaned.zip
Processing file : ANC_2018-2025_cleaned.zip
Processing file : ASE_2018-2025_cleaned.zip
Processing file : ATL_2018-2025_cleaned.zip
Processing file : AUS_2018-2025_cleaned.zip
Processing file : BDL_2018-2025_cleaned.zip
Processing file : BHM_2018-2025_cleaned.zip
Processing file : BIH_2018-2025_cleaned.zip
Processing file : BIL_2018-2025_cleaned.zip
Processing file : BLI_2018-2025_cleaned.zip
Processing file : BNA_2018-2025_cleaned.zip
Processing file : BOI_2018-2025_cleaned.zip
Processing file : BOS_2018-2025_cleaned.zip
Processing file : BTR_2018-2025_cleaned.zip
Processing file : BUF_2018-2025_cleaned.zip
Processing file : BUR_2018-2025_cleaned.zip
Processing file : BWI_2018-2025_cleaned.zip
Processing file : BZN_2018-2025_cleaned.zip
Processing file : CHS_2018-2025_cleaned.zip
Processing file : CID_2018-2025_cleaned.zip
Processing file : CLE_2018-2025_cleaned.zip
Processing file : CLT_2018-2025_

,DateTime,precip_accum_one_hour,air_temp,relative_humidity,wind_speed,sea_level_pressure,visibility,wind_gust,precip_accum_six_hour,ceiling,dew_point_temperature,ceiling_missing,wind_dir_sin,wind_dir_cos,wind_direction_interp,sea_level_pressure_missing,Airport
0,2019-01-01 00:49:00,0.0,51.80,18.60,4.61,NaN,10.0,0.0,0.0,35000,10.40,1,1.000000,6.123234e-17,90.0,1,LAX
1,2019-01-01 00:53:00,0.0,51.08,18.80,4.61,1015.3,10.0,0.0,0.0,35000,10.04,1,0.984808,-1.736482e-01,100.0,0,LAX
2,2019-01-01 01:53:00,0.0,51.98,18.18,0.00,1016.0,10.0,0.0,0.0,35000,10.04,1,0.000000,1.000000e+00,0.0,0,LAX
3,2019-01-01 02:53:00,0.0,48.92,21.20,5.75,1015.9,10.0,0.0,0.0,35000,10.94,1,0.173648,-9.848078e-01,170.0,0,LAX
4,2019-01-01 03:53:00,0.0,42.98,36.13,3.44,1016.1,10.0,0.0,0.0,35000,17.96,1,0.766044,-6.427876e-01,130.0,0,LAX


Destination Weather DateTime       datetime64[ns]
dest_precip_accum_one_hour                float64
dest_precip_accum_six_hour                float64
dest_air_temp                             float64
dest_dew_point_temperature                float64
dest_relative_humidity                    float64
dest_wind_speed                           float64
dest_wind_gust                            float64
dest_visibility                           float64
dest_ceiling                                int64
dest_sea_level_pressure                   float64
dest_ceiling_missing                        int64
dest_wind_dir_sin                         float64
dest_wind_dir_cos                         float64
dest_wind_direction_interp                float64
dest_sea_level_pressure_missing             int64
Airport                                    object
dtype: object


,Destination Weather DateTime,dest_precip_accum_one_hour,dest_precip_accum_six_hour,dest_air_temp,dest_dew_point_temperature,dest_relative_humidity,dest_wind_speed,dest_wind_gust,dest_visibility,dest_ceiling,dest_sea_level_pressure,dest_ceiling_missing,dest_wind_dir_sin,dest_wind_dir_cos,dest_wind_direction_interp,dest_sea_level_pressure_missing,Airport
0,2019-01-01 00:22:00,0.001,0.0,66.02,62.96,88.2,9.22,0.0,1.25,2100,1015.561667,0,-0.642788,-7.660444e-01,220.0,1,ABQ
1,2019-01-01 00:53:00,0.010,0.0,66.02,62.96,88.2,4.61,0.0,1.75,2300,1015.579592,0,-0.866025,-5.000000e-01,240.0,1,ABQ
2,2019-01-01 01:04:00,0.010,0.0,66.02,62.96,88.2,9.22,0.0,2.00,3000,1015.585952,0,-0.939693,-3.420201e-01,250.0,1,ABQ
3,2019-01-01 01:13:00,0.010,0.0,66.02,62.96,88.2,9.22,0.0,4.00,3300,1015.591156,0,-0.984808,-1.736482e-01,260.0,1,ABQ
4,2019-01-01 01:20:00,0.010,0.0,66.20,62.96,88.2,10.36,0.0,4.00,2800,1015.595204,0,-1.000000,-1.836970e-16,270.0,1,ABQ


## Read Aircraft Data

In [874]:
# read aircraft data
dtype={'TYPE ENGINE': object, 'LAST ACTION DATE': object}
aircraft_master_df = unzip_to_df(aircraft_folder + aircraft_zip_file, aircraft_master_file, dtype=dtype)
aircraft_master_df = aircraft_master_df.rename({'MFR MDL CODE': 'CODE', 'N-NUMBER': 'Tail Number'}, axis=1)

# drop columns we know we don't need
aircraft_master_df = aircraft_master_df.drop(['SERIAL NUMBER', 'CERTIFICATION', 'TYPE AIRCRAFT',
                                              'TYPE REGISTRANT', 'NAME', 'STREET', 'STREET2', 'CITY', 'STATE', 'ZIP CODE', 'REGION', 'COUNTY', 'COUNTRY', 
                                              'OTHER NAMES(1)', 'OTHER NAMES(2)', 'OTHER NAMES(3)', 'OTHER NAMES(4)','OTHER NAMES(5)',
                                              'STATUS CODE', 'MODE S CODE', 
                                              'FRACT OWNER', 'KIT MFR', ' KIT MODEL', 'MODE S CODE HEX', 'UNIQUE ID', 
                                              'Unnamed: 34'], axis=1)
print(f'{aircraft_master_df.dtypes}\n')
display(aircraft_master_df.head())
print(f'\nNumber of records: {aircraft_master_df.size}')

Tail Number         object
CODE                object
ENG MFR MDL         object
YEAR MFR            object
LAST ACTION DATE    object
CERT ISSUE DATE     object
TYPE ENGINE         object
AIR WORTH DATE      object
EXPIRATION DATE     object
dtype: object



,Tail Number,CODE,ENG MFR MDL,YEAR MFR,LAST ACTION DATE,CERT ISSUE DATE,TYPE ENGINE,AIR WORTH DATE,EXPIRATION DATE
0,100,7100510,17003,1940,20230122,20050506,1,19540430,20270430
1,10000,2130004,,,20240823,20240823,1,,20310831
2,10001,9601202,67007,1928,20230718,20190227,1,,20290228
3,10004,2072738,,,20230722,20130312,2,,20290331
4,10006,1152020,17026,1955,20230421,19980826,1,19710909,20280229



Number of records: 2706885


In [875]:
aircraft_reference_df = unzip_to_df(aircraft_folder + aircraft_zip_file, aircraft_reference_file, dtype={'BUILD-CERT-IND': 'Int64'})
aircraft_reference_df = aircraft_reference_df.drop(['TC-DATA-SHEET', 'TC-DATA-HOLDER',
                                                    'Unnamed: 13'], axis=1)
print(f'{aircraft_reference_df.dtypes}\n')
display(aircraft_reference_df.head())
print(f'\nNumber of records: {aircraft_reference_df.size}')

CODE              object
MFR               object
MODEL             object
TYPE-ACFT         object
TYPE-ENG           int64
AC-CAT             int64
BUILD-CERT-IND     Int64
NO-ENG             int64
NO-SEATS           int64
AC-WEIGHT         object
SPEED              int64
dtype: object



,CODE,MFR,MODEL,TYPE-ACFT,TYPE-ENG,AC-CAT,BUILD-CERT-IND,NO-ENG,NO-SEATS,AC-WEIGHT,SPEED
0,0020901,AAR AIRLIFT GROUP INC,UH-60A,6,3,1,0,2,15,CLASS 3,0
1,0030109,EXLINE ACE-C,ACE-C,4,1,1,1,1,1,CLASS 1,82
2,003010D,DELEBAUGH,P,4,1,1,1,1,1,CLASS 1,82
3,003010H,DAL PORTO,BABY ACE D,4,1,1,1,1,1,CLASS 1,82
4,003010P,DUNN,BABY ACE,4,1,1,1,1,1,CLASS 1,82



Number of records: 1017973


In [876]:
aircraft_df = aircraft_master_df.merge(aircraft_reference_df, how='left', on='CODE')
aircraft_df.head()

,Tail Number,CODE,ENG MFR MDL,YEAR MFR,LAST ACTION DATE,CERT ISSUE DATE,TYPE ENGINE,AIR WORTH DATE,EXPIRATION DATE,MFR,MODEL,TYPE-ACFT,TYPE-ENG,AC-CAT,BUILD-CERT-IND,NO-ENG,NO-SEATS,AC-WEIGHT,SPEED
0,100,7100510,17003,1940,20230122,20050506,1,19540430,20270430,PIPER,J3C-65,4,1,1,0,1,2,CLASS 1,67
1,10000,2130004,,,20240823,20240823,1,,20310831,CIRRUS DESIGN CORP,SR22T,4,1,1,0,1,5,CLASS 1,0
2,10001,9601202,67007,1928,20230718,20190227,1,,20290228,WACO,ASO,4,1,1,0,1,3,CLASS 1,79
3,10004,2072738,,,20230722,20130312,2,,20290331,CESSNA,T182T,4,2,1,0,1,4,CLASS 1,0
4,10006,1152020,17026,1955,20230421,19980826,1,19710909,20280229,BEECH,D-45 (T-34B),4,1,1,0,1,4,CLASS 1,0


In [877]:
# strip white space from all columns
aircraft_df = aircraft_df.iloc[:].map(lambda x: x.strip() if isinstance(x, str) else x)
aircraft_df['MFR'].unique()

array(['PIPER', 'CIRRUS DESIGN CORP', 'WACO', ..., 'SHAY GREGORIE',
       'ATEC', 'HARRELSON SUSAN E'], dtype=object)

In [878]:
# re-code tail numbers for consistency with delays file
aircraft_df['Tail Number'] = 'N' + aircraft_df['Tail Number']
aircraft_df['Tail Number'].unique()

array(['N100', 'N10000', 'N10001', ..., 'N9ZT', 'N9ZU', 'N9ZX'],
      dtype=object)

## Read Delays Data

In [879]:
# function to read delays file
def read_delays_file(folder, file):
    delays_df = pd.read_csv(f'{folder}{file}')
    delays_df['DateTime'] = pd.to_datetime(delays_df['Date (MM/DD/YYYY)'] + ' ' + delays_df['Scheduled departure time'], format='%m/%d/%Y %H:%M')
    delays_df = delays_df.drop(['Wheels-off time', 'Taxi-Out time (Minutes)', 'Delay Carrier (Minutes)',
                                'Delay Weather (Minutes)', 'Delay National Aviation System (Minutes)', 'Delay Security (Minutes)',
                                'Delay Late Aircraft Arrival (Minutes)'], axis=1)
    delays_df = delays_df.rename({'Date (MM/DD/YYYY)': 'Date',
                                  'Scheduled departure time': 'Scheduled Departure Time',
                                  'Actual departure time': 'Actual Departure Time',
                                  'Scheduled elapsed time (Minutes)': 'Scheduled Elapsed Time',
                                  'Actual elapsed time (Minutes)': 'Actual Elapsed Time',
                                  'Departure delay (Minutes)': 'Departure Delay'}, axis=1)
    
    return delays_df

In [880]:
delays_list = [read_delays_file(delays_folder, delays_file) for delays_file in delays_files]
delays_df = pd.concat(delays_list)
delays_df = delays_df.reset_index(drop=True)
print(len(delays_df))
delays_df.tail()

894689


,Carrier Code,Date,Flight Number,Tail Number,Destination Airport,Scheduled Departure Time,Actual Departure Time,Scheduled Elapsed Time,Actual Elapsed Time,Departure Delay,DateTime
894684,WN,12/31/2024,4297,N8564Z,PHX,14:25,14:47,85,75,22,2024-12-31 14:25:00
894685,WN,12/31/2024,4334,N8930S,LAS,17:05,17:07,70,63,2,2024-12-31 17:05:00
894686,WN,12/31/2024,4427,N291WN,SFO,14:50,14:53,85,75,3,2024-12-31 14:50:00
894687,WN,12/31/2024,4598,N8751R,DEN,18:20,18:15,145,125,-5,2024-12-31 18:20:00
894688,WN,12/31/2024,4843,N8634A,OAK,11:20,11:30,90,72,10,2024-12-31 11:20:00


# Merge dataframes

## Calculate Arrival DateTime

In [881]:
delays_df['Scheduled Arrival DateTime'] = delays_df['DateTime'] + pd.to_timedelta(delays_df['Scheduled Elapsed Time'], unit='m')
delays_df[['DateTime', 'Scheduled Elapsed Time', 'Scheduled Arrival DateTime']].head()

,DateTime,Scheduled Elapsed Time,Scheduled Arrival DateTime
0,2020-01-01 08:00:00,319,2020-01-01 13:19:00
1,2020-01-01 11:00:00,325,2020-01-01 16:25:00
2,2020-01-01 21:30:00,317,2020-01-02 02:47:00
3,2020-01-01 19:30:00,240,2020-01-01 23:30:00
4,2020-01-01 23:00:00,316,2020-01-02 04:16:00


## Merge with aircraft data

In [882]:
delays_df = delays_df.merge(aircraft_df, on='Tail Number', how='left')
delays_df.tail()

,Carrier Code,Date,Flight Number,Tail Number,Destination Airport,Scheduled Departure Time,Actual Departure Time,Scheduled Elapsed Time,Actual Elapsed Time,Departure Delay,DateTime,Scheduled Arrival DateTime,CODE,ENG MFR MDL,YEAR MFR,LAST ACTION DATE,CERT ISSUE DATE,TYPE ENGINE,AIR WORTH DATE,EXPIRATION DATE,MFR,MODEL,TYPE-ACFT,TYPE-ENG,AC-CAT,BUILD-CERT-IND,NO-ENG,NO-SEATS,AC-WEIGHT,SPEED
894684,WN,12/31/2024,4297,N8564Z,PHX,14:25,14:47,85,75,22,2024-12-31 14:25:00,2024-12-31 15:50:00,1384414,13100,2018,20230428,20180320,5,20180224,20280331,BOEING,737-800,5,5.0,1.0,0.0,2.0,175.0,CLASS 3,0.0
894685,WN,12/31/2024,4334,N8930S,LAS,17:05,17:07,70,63,2,2024-12-31 17:05:00,2024-12-31 18:15:00,13844FZ,13120,2024,20241025,20241025,5,20241021,20311031,BOEING,737-8,5,5.0,1.0,1.0,2.0,175.0,CLASS 3,0.0
894686,WN,12/31/2024,4427,N291WN,SFO,14:50,14:53,85,75,3,2024-12-31 14:50:00,2024-12-31 16:15:00,1384404,13044,2007,20230512,20070927,5,20070909,20280531,BOEING,737-7H4,5,5.0,1.0,0.0,2.0,143.0,CLASS 3,0.0
894687,WN,12/31/2024,4598,N8751R,DEN,18:20,18:15,145,125,-5,2024-12-31 18:20:00,2024-12-31 20:45:00,13844FZ,13128,2021,20230616,20211008,5,20210921,20281031,BOEING,737-8,5,5.0,1.0,1.0,2.0,175.0,CLASS 3,0.0
894688,WN,12/31/2024,4843,N8634A,OAK,11:20,11:30,90,72,10,2024-12-31 11:20:00,2024-12-31 12:50:00,138440A,13804,2014,20230304,20140708,5,20140613,20270731,BOEING,737-8H4,5,5.0,1.0,0.0,2.0,143.0,CLASS 3,0.0


## Merge with weather data

#### Check for discrepancies between Destination Airport and destination weather Airport

In [883]:
dest_airports = list(delays_df['Destination Airport'].unique())
dest_weather_airports = list(weather_destination_df['Airport'].unique())

print(dest_airports)
print(len(dest_airports))

print(dest_weather_airports)
print(len(dest_weather_airports))

np.setdiff1d(dest_airports, dest_weather_airports)

['JFK', 'ORD', 'HNL', 'PHL', 'PHX', 'KOA', 'DCA', 'DFW', 'OGG', 'BOS', 'LIH', 'CLT', 'IAD', 'MIA', 'IAH', 'IND', 'SFO', 'AUS', 'CMH', 'STL', 'LAS', 'ATL', 'BDL', 'BNA', 'DEN', 'SAT', 'EGE', 'RDU', 'MCO', 'MEM', 'MSY', 'SDF', 'OMA', 'RNO', 'FLL', 'TPA', 'SEA', 'SJC', 'CVG', 'OKC', 'SLC', 'ELP', 'DTW', 'SBN', 'PDX', 'TUL', 'ANC', 'EWR', 'BWI', 'RSW', 'GEG', 'BOI', 'PAE', 'DAL', 'RDM', 'EUG', 'BUF', 'PBI', 'CHS', 'JAX', 'RIC', 'MTJ', 'BZN', 'MFR', 'MSO', 'STS', 'FAT', 'FCA', 'SAN', 'MSP', 'MCI', 'BHM', 'LGA', 'SMF', 'MSN', 'ABQ', 'MKE', 'CID', 'DSM', 'SHV', 'BLI', 'SGF', 'GRR', 'PVU', 'XNA', 'BIL', 'LIT', 'GJT', 'MFE', 'PSC', 'ICT', 'FSD', 'FAR', 'JAC', 'IDA', 'RAP', 'RFD', 'LRD', 'SCK', 'ATW', 'CLE', 'PIT', 'OAK', 'TUS', 'ASE', 'SUN', 'SGU', 'PRC', 'MRY', 'COS', 'PSP', 'SBA', 'SBP', 'HDN', 'MMH', 'RDD', 'ACV', 'BIH', 'DRO', 'SAF', 'FLG', 'BUR', 'ITO', 'BTR', 'SNA', 'MDW', 'HOU']
127
['ABQ', 'ACV', 'ANC', 'ASE', 'ATL', 'AUS', 'BDL', 'BHM', 'BIH', 'BIL', 'BLI', 'BNA', 'BOI', 'BOS', 'BTR', 

array(['ATW', 'FCA', 'SLC', 'SUN'], dtype='<U3')

In [884]:
# Data needs to be sorted to use merge_asof
delays_df = delays_df.sort_values('DateTime')
weather_df = weather_df.sort_values('DateTime')

# Use merge_asof to merge with the nearest time
delays_df = pd.merge_asof(delays_df, weather_df,
                          left_on=['DateTime'],
                          right_on=['DateTime'],
                          direction='backward')

delays_df = delays_df.drop(['Airport'], axis=1)

# Re-sort to merge destination weather values
delays_df = delays_df.sort_values('Scheduled Arrival DateTime')
weather_destination_df = weather_destination_df.sort_values('Destination Weather DateTime')

delays_df = pd.merge_asof(delays_df, weather_destination_df,
                          left_on=['Scheduled Arrival DateTime'],
                          right_on=['Destination Weather DateTime'],
                          left_by=['Destination Airport'],
                          right_by=['Airport'],
                          direction='backward')
delays_df.head(10)

,Carrier Code,Date,Flight Number,Tail Number,Destination Airport,Scheduled Departure Time,Actual Departure Time,Scheduled Elapsed Time,Actual Elapsed Time,Departure Delay,DateTime,Scheduled Arrival DateTime,CODE,ENG MFR MDL,YEAR MFR,LAST ACTION DATE,CERT ISSUE DATE,TYPE ENGINE,AIR WORTH DATE,EXPIRATION DATE,MFR,MODEL,TYPE-ACFT,TYPE-ENG,AC-CAT,BUILD-CERT-IND,NO-ENG,NO-SEATS,AC-WEIGHT,SPEED,precip_accum_one_hour,air_temp,relative_humidity,wind_speed,sea_level_pressure,visibility,wind_gust,precip_accum_six_hour,ceiling,dew_point_temperature,ceiling_missing,wind_dir_sin,wind_dir_cos,wind_direction_interp,sea_level_pressure_missing,Destination Weather DateTime,dest_precip_accum_one_hour,dest_precip_accum_six_hour,dest_air_temp,dest_dew_point_temperature,dest_relative_humidity,dest_wind_speed,dest_wind_gust,dest_visibility,dest_ceiling,dest_sea_level_pressure,dest_ceiling_missing,dest_wind_dir_sin,dest_wind_dir_cos,dest_wind_direction_interp,dest_sea_level_pressure_missing,Airport
0,F9,01/01/2020,402,N323FR,DEN,00:59,00:46,147,145,-13,2020-01-01 00:59:00,2020-01-01 03:26:00,3940325,13110,2017,20230407,20171204,5,20171205,20271231,AIRBUS,A320-251N,5,5.0,1.0,0.0,2.0,190.0,CLASS 3,0.0,0.0,59.00,37.47,6.91,1017.2,10.0,0.0,0.0,35000,33.08,1,0.342020,0.939693,20.0,0,2020-01-01 02:53:00,0.000,0.0,69.08,57.92,64.32,5.75,0.0,10.0,4700.0,1017.7,0.0,-0.766044,-0.642788,230.0,0.0,DEN
1,UA,01/01/2020,1990,N847UA,IAH,00:57,00:50,183,182,-7,2020-01-01 00:57:00,2020-01-01 04:00:00,3930317,34601,2001,20230616,20181030,4,20020115,20281031,AIRBUS INDUSTRIE,A319-131,5,4.0,1.0,0.0,2.0,179.0,CLASS 3,0.0,0.0,59.00,37.47,6.91,1017.2,10.0,0.0,0.0,35000,33.08,1,0.342020,0.939693,20.0,0,2020-01-01 03:53:00,0.001,0.0,68.00,59.00,72.97,5.75,0.0,5.0,3600.0,1017.5,0.0,-0.866025,-0.500000,240.0,0.0,IAH
2,DL,01/01/2020,1969,N543US,MSP,00:40,00:29,209,219,-11,2020-01-01 00:40:00,2020-01-01 04:09:00,1384970,52117,1996,20230804,20160503,5,19960515,20290531,BOEING,757-251,5,5.0,1.0,0.0,2.0,178.0,CLASS 3,0.0,0.0,57.02,40.23,5.75,1017.5,10.0,0.0,0.0,35000,33.08,1,0.984808,0.173648,80.0,0,2020-01-01 03:53:00,0.001,0.0,68.00,59.00,72.97,5.75,0.0,5.0,3600.0,1017.5,0.0,-0.866025,-0.500000,240.0,0.0,MSP
3,AA,01/01/2020,1876,N996AN,DFW,01:30,01:48,179,174,18,2020-01-01 01:30:00,2020-01-01 04:29:00,3940032,34611,2016,20230922,20161220,5,20161220,20291231,AIRBUS,A321-231,5,5.0,1.0,0.0,2.0,379.0,CLASS 3,0.0,0.0,59.00,37.47,6.91,1017.2,10.0,0.0,0.0,35000,33.08,1,0.342020,0.939693,20.0,0,2020-01-01 03:53:00,0.001,0.0,68.00,59.00,72.97,5.75,0.0,5.0,3600.0,1017.5,0.0,-0.866025,-0.500000,240.0,0.0,DFW
4,UA,01/01/2020,910,N38479,ORD,00:48,00:40,234,238,-8,2020-01-01 00:48:00,2020-01-01 04:42:00,138488H,13100,2016,20230922,20161223,5,20161222,20291231,BOEING,737-900ER,5,5.0,1.0,0.0,2.0,222.0,CLASS 3,0.0,0.0,57.02,40.23,5.75,1017.5,10.0,0.0,0.0,35000,33.08,1,0.984808,0.173648,80.0,0,2020-01-01 03:53:00,0.001,0.0,68.00,59.00,72.97,5.75,0.0,5.0,3600.0,1017.5,0.0,-0.866025,-0.500000,240.0,0.0,ORD
5,AA,01/01/2020,1189,N198UW,CLT,00:30,00:41,278,255,11,2020-01-01 00:30:00,2020-01-01 05:08:00,3940005,13006,2013,20230701,20151230,5,20130213,20281231,AIRBUS,A321-211,5,5.0,1.0,0.0,2.0,199.0,CLASS 3,0.0,0.0,57.02,40.23,5.75,1017.5,10.0,0.0,0.0,35000,33.08,1,0.984808,0.173648,80.0,0,2020-01-01 04:53:00,0.001,0.0,69.08,59.00,68.61,6.91,0.0,10.0,35000.0,1017.4,1.0,-0.866025,-0.500000,240.0,0.0,CLT
6,UA,01/01/2020,1165,N14121,EWR,00:15,00:13,307,298,-2,2020-01-01 00:15:00,2020-01-01 05:22:00,1384958,54555,1997,20230526,20120720,5,19970729,20280731,BOEING,757-224,5,5.0,1.0,0.0,2.0,178.0,CLASS 3,0.0,0.0,57.02,40.23,5.75,1017.5,10.0,0.0,0.0,35000,33.08,1,0.984808,0.173648,80.0,0,2020-01-01 04:53:00,0.001,0.0,69.08,59.00,68.61,6.91,0.0,10.0,35000.0,1017.4,1.0,-0.866025,-0.500000,240.0,0.0,EWR
7,AA,01/01/2020,362,N193UW,ORD,01:20,01:18,244,259,-2,2020-01-01 01:20:00,2020-01-01 05:24:00,3940005,13006,2008,20230701,20151230,5,20080718,20281231,AIRBUS,A321-211,5,5.0,1.0,0.0,2.0,199.0,CLASS 3,0.0,0.0,59.00,37.

In [885]:
delays_df.columns

Index(['Carrier Code', 'Date', 'Flight Number', 'Tail Number',
       'Destination Airport', 'Scheduled Departure Time',
       'Actual Departure Time', 'Scheduled Elapsed Time',
       'Actual Elapsed Time', 'Departure Delay', 'DateTime',
       'Scheduled Arrival DateTime', 'CODE', 'ENG MFR MDL', 'YEAR MFR',
       'LAST ACTION DATE', 'CERT ISSUE DATE', 'TYPE ENGINE', 'AIR WORTH DATE',
       'EXPIRATION DATE', 'MFR', 'MODEL', 'TYPE-ACFT', 'TYPE-ENG', 'AC-CAT',
       'BUILD-CERT-IND', 'NO-ENG', 'NO-SEATS', 'AC-WEIGHT', 'SPEED',
       'precip_accum_one_hour', 'air_temp', 'relative_humidity', 'wind_speed',
       'sea_level_pressure', 'visibility', 'wind_gust',
       'precip_accum_six_hour', 'ceiling', 'dew_point_temperature',
       'ceiling_missing', 'wind_dir_sin', 'wind_dir_cos',
       'wind_direction_interp', 'sea_level_pressure_missing',
       'Destination Weather DateTime', 'dest_precip_accum_one_hour',
       'dest_precip_accum_six_hour', 'dest_air_temp',
       'dest_dew_

# Check value counts of suspect columns to see which to drop or condense

In [886]:
# Check value counts of suspect columns to see which to drop or condense
columns = ['CODE', 'ENG MFR MDL', 'YEAR MFR', 'TYPE ENGINE', 'MFR', 'MODEL', 'TYPE-ACFT', 'TYPE-ENG',
           'AC-CAT', 'NO-ENG', 'NO-SEATS', 'AC-WEIGHT', 'SPEED', 'BUILD-CERT-IND', 'Destination Airport']

for column in columns:
    print(delays_df[column].value_counts())
    print(f'Unique values = {delays_df[column].nunique()}\n\n')

CODE
3260415    80147
3940032    68567
326041A    52424
3940051    38654
1384404    36796
           ...  
4470728        3
4220011        3
13844D1        1
7320001        1
1385232        1
Name: count, Length: 139, dtype: int64
Unique values = 139


ENG MFR MDL
30061    166361
13802     86827
34611     85261
13078     63323
13120     39760
          ...  
17042         5
33940         3
03002         3
52250         1
30050         1
Name: count, Length: 115, dtype: int64
Unique values = 115


YEAR MFR
2018    89970
2017    65542
2014    58422
2016    57467
2021    51911
2019    51125
2022    41553
2015    40627
2001    32854
2013    27595
2020    27170
2002    23500
2000    23304
2023    22399
2004    21294
2003    21169
2012    20782
2005    18411
1999    16209
2009    16044
2006    15199
2008    13872
2007    12363
        12298
2010    12133
1998    11782
2011    10315
1996     9166
1997     6818
2024     4539
1995     4438
1992     3863
1991     3243
1994     3175
1990     2566

# Keep only major commercial aircraft

In [887]:
# Keep only major commercial aircraft (TYPE-ACFT == '5', fixed wing multi engine)
delays_df = delays_df[delays_df['TYPE-ACFT'] == '5']
delays_df['TYPE-ACFT'].value_counts()

TYPE-ACFT
5    854617
Name: count, dtype: int64

In [888]:
# Remove columns with (predominantly) only 1 value
# But should we drop TYPE-ACFT of 6 (rotorcraft/helicopter), 2 (balloon), and even 4 (fixed wing single engine)?
# Similarly should we drop TYPE ENGINE other than 5 (turbo-fan) and 4 (turbo-jet)?
# But then we can't accurately check congestion

delays_df = delays_df.drop(['TYPE-ACFT', 'AC-CAT', 'NO-ENG', 'AC-WEIGHT', 'SPEED'], axis=1)
delays_df.columns
print(f'Number of columns:   {len(delays_df.columns)}')

Number of columns:   57


In [889]:
def condense_column_values(series, min, max, replacement_string='Other'):
  # series is a series/dataframe column
  # cutoff_value is an integer representing the target value_count

  min_cutoffs = series.value_counts() >= min
  max_cutoffs = series.value_counts() < max
  values_to_condense = min_cutoffs[min_cutoffs & max_cutoffs].index

  condensed_series = series.copy()

  # Replace values below the cutoff with repalcement_string
  for v in values_to_condense:
    condensed_series = condensed_series.replace(v, replacement_string)

  return condensed_series

# Consolidate manufacturers

In [890]:
# Get values and counts of manufacturers
delays_df['MFR'].value_counts()

MFR
BOEING                            356774
AIRBUS                            215325
EMBRAER S A                       132571
AIRBUS INDUSTRIE                   52809
BOMBARDIER INC                     39472
YABORA INDUSTRIA AERONAUTICA S     29239
AIRBUS SAS                         13889
AIRBUS S A S                        4542
EMBRAER                             3512
AIRBUS CANADA LP                    2011
AIRBUS CANADA LTD PTNRSP            1813
EMBRAER-EMPRESA BRASILEIRA DE       1705
C SERIES AIRCRAFT LTD PTNRSP         946
CESSNA                                 6
HAWKER BEECHCRAFT CORP                 3
Name: count, dtype: int64

In [891]:
# Airbus
replace_list = ['AIRBUS', 'AIRBUS INDUSTRIE', 'AIRBUS SAS', 'AIRBUS S A S']
delays_df['MFR'] = delays_df['MFR'].replace(replace_list, 'AIRBUS')

# Airbus Canada LP -- but should we merge this with Airbus above?
replace_list = ['AIRBUS CANADA LP', 'AIRBUS CANADA LTD PTNRSP', 'C SERIES AIRCRAFT LTD PTNRSP']
delays_df['MFR'] = delays_df['MFR'].replace(replace_list, 'AIRBUS CANADA LP')

# Embraer
replace_list = ['EMBRAER', 'EMBRAER S A','EMBRAER-EMPRESA BRASILEIRA DE']
delays_df['MFR'] = delays_df['MFR'].replace(replace_list, 'EMBRAER')

# Combine low-count manufacturers into "OTHER"
delays_df['MFR'] = condense_column_values(delays_df['MFR'], 0, 1000, 'OTHER')

delays_df['MFR'].value_counts()

MFR
BOEING                            356774
AIRBUS                            286565
EMBRAER                           137788
BOMBARDIER INC                     39472
YABORA INDUSTRIA AERONAUTICA S     29239
AIRBUS CANADA LP                    4770
OTHER                                  9
Name: count, dtype: int64

# Consolidate aircraft models

In [892]:
delays_df['MODEL'] = condense_column_values(delays_df['MODEL'], 0, 1000, 'OTHER')
delays_df['MODEL'].value_counts()

MODEL
ERJ 170-200 LR    111303
A321-231           86595
ERJ 170-200 LL     55724
A321-253NX         38665
A320-232           38282
                   ...  
737-7CT             1348
767-322             1194
A319-131            1093
737-8EH             1088
737-752             1085
Name: count, Length: 65, dtype: int64

In [893]:
# Check if these columns have the same information
print(delays_df['TYPE ENGINE'].value_counts())
print(delays_df['TYPE-ENG'].value_counts())

TYPE ENGINE
5    848202
4      6409
1         6
Name: count, dtype: int64
TYPE-ENG
5.0    848202
4.0      6409
1.0         6
Name: count, dtype: int64


In [894]:
# Those columns are redundant, so Drop TYPE-ENG
delays_df = delays_df.drop('TYPE-ENG', axis=1)

# Consolidate destination airports

In [895]:
delays_df['Destination Airport'].value_counts()

Destination Airport
SFO    50831
LAS    46416
JFK    42020
DEN    32844
PHX    31670
       ...  
BHM        3
BTR        1
CID        1
DSM        1
BUR        1
Name: count, Length: 110, dtype: int64

In [896]:
cutoff = 100
cutoffs = delays_df['Destination Airport'].value_counts() < cutoff
filter_list = list(cutoffs[cutoffs].index)
filter = delays_df['Destination Airport'].isin(filter_list)
print(delays_df[filter]['Destination Airport'].value_counts())

Destination Airport
PSC    96
MSN    94
MMH    91
BIH    89
DRO    74
LGA    70
GJT    14
FLG    12
SAF    11
SBN     4
BHM     3
BTR     1
CID     1
DSM     1
BUR     1
Name: count, dtype: int64


In [897]:
delays_df['Destination Airport'] = condense_column_values(delays_df['Destination Airport'], 0, 100, 'OTHER')
delays_df['Destination Airport'].value_counts()

Destination Airport
SFO    50831
LAS    46416
JFK    42020
DEN    32844
PHX    31670
       ...  
MTJ      174
SGU      173
SCK      138
RIC      116
MKE      102
Name: count, Length: 96, dtype: int64

# Create bins for delay time

In [898]:
delay_times = delays_df['Departure Delay'].describe()
delay_times

count    854617.000000
mean          8.850076
std          46.284626
min         -47.000000
25%          -6.000000
50%          -2.000000
75%           5.000000
max        2640.000000
Name: Departure Delay, dtype: float64

In [899]:
# Create arbitrary bins
delay_time_bins = [delay_times['min'].astype('int64'), 0, 1, 11, 31, 61, delay_times['max'].astype('int64')+1]
delay_time_labels = ['Early', 'On Time',
                     f'{delay_time_bins[2]}-{delay_time_bins[3]-1} min',
                     f'{delay_time_bins[3]}-{delay_time_bins[4]-1} min',
                     f'{delay_time_bins[4]}-{delay_time_bins[5]-1} min',
                     f'{delay_time_bins[5]}+ min']

print(delay_time_bins)
print(delay_time_labels)

delays_df['Delay Bin'] = pd.cut(delays_df['Departure Delay'], bins=delay_time_bins, labels=delay_time_labels, right=False)
delays_df['Delay Bin'].value_counts().sort_index()

[-47, 0, 1, 11, 31, 61, 2641]
['Early', 'On Time', '1-10 min', '11-30 min', '31-60 min', '61+ min']


Delay Bin
Early        516648
On Time       53251
1-10 min     118756
11-30 min     79669
31-60 min     41887
61+ min       44406
Name: count, dtype: int64

# Reformat column values for readability

In [900]:
# Reformat column values
delays_df['TYPE ENGINE'] = delays_df['TYPE ENGINE'].replace({'5': 'Turbo-Fan',
                                                             '4': 'Turbo-Jet',
                                                             '1': 'Reciprocating' })

delays_df['BUILD-CERT-IND'] = delays_df['BUILD-CERT-IND'].replace({0: True,
                                                                   1: False})

delays_df['BUILD-CERT-IND'].value_counts()


C:\Users\David\AppData\Local\Temp\ipykernel_3800\163929967.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  delays_df['BUILD-CERT-IND'] = delays_df['BUILD-CERT-IND'].replace({0: True,


BUILD-CERT-IND
True     805193
False     49424
Name: count, dtype: int64

In [901]:
delays_df.columns

Index(['Carrier Code', 'Date', 'Flight Number', 'Tail Number',
       'Destination Airport', 'Scheduled Departure Time',
       'Actual Departure Time', 'Scheduled Elapsed Time',
       'Actual Elapsed Time', 'Departure Delay', 'DateTime',
       'Scheduled Arrival DateTime', 'CODE', 'ENG MFR MDL', 'YEAR MFR',
       'LAST ACTION DATE', 'CERT ISSUE DATE', 'TYPE ENGINE', 'AIR WORTH DATE',
       'EXPIRATION DATE', 'MFR', 'MODEL', 'BUILD-CERT-IND', 'NO-SEATS',
       'precip_accum_one_hour', 'air_temp', 'relative_humidity', 'wind_speed',
       'sea_level_pressure', 'visibility', 'wind_gust',
       'precip_accum_six_hour', 'ceiling', 'dew_point_temperature',
       'ceiling_missing', 'wind_dir_sin', 'wind_dir_cos',
       'wind_direction_interp', 'sea_level_pressure_missing',
       'Destination Weather DateTime', 'dest_precip_accum_one_hour',
       'dest_precip_accum_six_hour', 'dest_air_temp',
       'dest_dew_point_temperature', 'dest_relative_humidity',
       'dest_wind_speed', 'd

# Reformat column names for readability

In [902]:
# Reformat column names
delays_df = delays_df.rename(columns={
    'CODE': 'Manufacturer Code',
    'ENG MFR MDL': 'Engine Manufacturer Code',
    'YEAR MFR': 'Year of Manufacture',
    'LAST ACTION DATE': 'Certificate Last Activity Date',    
    'CERT ISSUE DATE': 'Certificate Issue Date',
    'TYPE ENGINE': 'Type of Engine',
    'AIR WORTH DATE': 'Air Worthiness Date',
    'EXPIRATION DATE': 'Certificate Expiration Date',
    'MFR': 'Manufacturer',
    'MODEL': 'Model',
    'BUILD-CERT-IND': 'Builder Type Certificated',
    'NO-SEATS': 'Number of Seats',
    'precip_accum_one_hour': 'Precipitation Accumulation One Hour',
    'precip_accum_six_hour': 'Precipitation Accumulation Six Hours',
    'air_temp': 'Air Temperature',
    'dew_point_temperature': 'Dew Point Temperature',
    'relative_humidity': 'Relative Humidity',
    'wind_speed': 'Wind Speed',
    'wind_direction': 'Wind Direction',
    'wind_gust': 'Wind Gust',
    'visibility': 'Visibility',
    'ceiling': 'Ceiling',
    'sea_level_pressure': 'Sea Level Pressure',
    'weather_cond_code': 'Weather Condition Code',
    'ceiling_missing': 'Ceiling Missing',
    'wind_direction_interp': 'Wind Direction Interpolation',
    'sea_level_pressure_missing': 'Sea Level Pressure Missing',
    'dest_precip_accum_one_hour': 'Destination Precipication Accumulation One Hour',
    'dest_precip_accum_six_hour': 'Destination Precipitation Six Hours',
    'dest_air_temp': 'Destination Air Temperature',
    'dest_dew_point_temperature': 'Destination Dew Point Temperature', 
    'dest_relative_humidity': 'Destination Relative Humidity',       
    'dest_wind_speed': 'Destination Wind Speed',
    'dest_wind_gust': 'Destination Wind Gust',
    'dest_visibility': 'Destination Visibility',
    'dest_ceiling': 'Destination Ceiling',
    'dest_sea_level_pressure': 'Destination Sea Level Pressure',
    'dest_ceiling_missing': 'Destination Ceiling Missing',
    'dest_wind_direction_interp': 'Destination Wind Direction Interpolation',
    'dest_sea_level_pressure_missing': 'Destination Sea Level Pressure Missing',
    'wind_dir_sin': 'Wind Direction (sin)',
    'wind_dir_cos': 'Wind Direction (cos)',
    'dest_wind_dir_sin': 'Destination Wind Direction (sin)',
    'dest_wind_dir_cos': 'Destination Wind Direction (cos)'
})
delays_df.columns

Index(['Carrier Code', 'Date', 'Flight Number', 'Tail Number',
       'Destination Airport', 'Scheduled Departure Time',
       'Actual Departure Time', 'Scheduled Elapsed Time',
       'Actual Elapsed Time', 'Departure Delay', 'DateTime',
       'Scheduled Arrival DateTime', 'Manufacturer Code',
       'Engine Manufacturer Code', 'Year of Manufacture',
       'Certificate Last Activity Date', 'Certificate Issue Date',
       'Type of Engine', 'Air Worthiness Date', 'Certificate Expiration Date',
       'Manufacturer', 'Model', 'Builder Type Certificated', 'Number of Seats',
       'Precipitation Accumulation One Hour', 'Air Temperature',
       'Relative Humidity', 'Wind Speed', 'Sea Level Pressure', 'Visibility',
       'Wind Gust', 'Precipitation Accumulation Six Hours', 'Ceiling',
       'Dew Point Temperature', 'Ceiling Missing', 'Wind Direction (sin)',
       'Wind Direction (cos)', 'Wind Direction Interpolation',
       'Sea Level Pressure Missing', 'Destination Weather DateTime

In [903]:
delays_df.head()

,Carrier Code,Date,Flight Number,Tail Number,Destination Airport,Scheduled Departure Time,Actual Departure Time,Scheduled Elapsed Time,Actual Elapsed Time,Departure Delay,DateTime,Scheduled Arrival DateTime,Manufacturer Code,Engine Manufacturer Code,Year of Manufacture,Certificate Last Activity Date,Certificate Issue Date,Type of Engine,Air Worthiness Date,Certificate Expiration Date,Manufacturer,Model,Builder Type Certificated,Number of Seats,Precipitation Accumulation One Hour,Air Temperature,Relative Humidity,Wind Speed,Sea Level Pressure,Visibility,Wind Gust,Precipitation Accumulation Six Hours,Ceiling,Dew Point Temperature,Ceiling Missing,Wind Direction (sin),Wind Direction (cos),Wind Direction Interpolation,Sea Level Pressure Missing,Destination Weather DateTime,Destination Precipication Accumulation One Hour,Destination Precipitation Six Hours,Destination Air Temperature,Destination Dew Point Temperature,Destination Relative Humidity,Destination Wind Speed,Destination Wind Gust,Destination Visibility,Destination Ceiling,Destination Sea Level Pressure,Destination Ceiling Missing,Destination Wind Direction (sin),Destination Wind Direction (cos),Destination Wind Direction Interpolation,Destination Sea Level Pressure Missing,Airport,Delay Bin
0,F9,01/01/2020,402,N323FR,DEN,00:59,00:46,147,145,-13,2020-01-01 00:59:00,2020-01-01 03:26:00,3940325,13110,2017,20230407,20171204,Turbo-Fan,20171205,20271231,AIRBUS,A320-251N,True,190.0,0.0,59.00,37.47,6.91,1017.2,10.0,0.0,0.0,35000,33.08,1,0.342020,0.939693,20.0,0,2020-01-01 02:53:00,0.000,0.0,69.08,57.92,64.32,5.75,0.0,10.0,4700.0,1017.7,0.0,-0.766044,-0.642788,230.0,0.0,DEN,Early
1,UA,01/01/2020,1990,N847UA,IAH,00:57,00:50,183,182,-7,2020-01-01 00:57:00,2020-01-01 04:00:00,3930317,34601,2001,20230616,20181030,Turbo-Jet,20020115,20281031,AIRBUS,A319-131,True,179.0,0.0,59.00,37.47,6.91,1017.2,10.0,0.0,0.0,35000,33.08,1,0.342020,0.939693,20.0,0,2020-01-01 03:53:00,0.001,0.0,68.00,59.00,72.97,5.75,0.0,5.0,3600.0,1017.5,0.0,-0.866025,-0.500000,240.0,0.0,IAH,Early
2,DL,01/01/2020,1969,N543US,MSP,00:40,00:29,209,219,-11,2020-01-01 00:40:00,2020-01-01 04:09:00,1384970,52117,1996,20230804,20160503,Turbo-Fan,19960515,20290531,BOEING,757-251,True,178.0,0.0,57.02,40.23,5.75,1017.5,10.0,0.0,0.0,35000,33.08,1,0.984808,0.173648,80.0,0,2020-01-01 03:53:00,0.001,0.0,68.00,59.00,72.97,5.75,0.0,5.0,3600.0,1017.5,0.0,-0.866025,-0.500000,240.0,0.0,MSP,Early
3,AA,01/01/2020,1876,N996AN,DFW,01:30,01:48,179,174,18,2020-01-01 01:30:00,2020-01-01 04:29:00,3940032,34611,2016,20230922,20161220,Turbo-Fan,20161220,20291231,AIRBUS,A321-231,True,379.0,0.0,59.00,37.47,6.91,1017.2,10.0,0.0,0.0,35000,33.08,1,0.342020,0.939693,20.0,0,2020-01-01 03:53:00,0.001,0.0,68.00,59.00,72.97,5.75,0.0,5.0,3600.0,1017.5,0.0,-0.866025,-0.500000,240.0,0.0,DFW,11-30 min
4,UA,01/01/2020,910,N38479,ORD,00:48,00:40,234,238,-8,2020-01-01 00:48:00,2020-01-01 04:42:00,138488H,13100,2016,20230922,20161223,Turbo-Fan,20161222,20291231,BOEING,737-900ER,True,222.0,0.0,57.02,40.23,5.75,1017.5,10.0,0.0,0.0,35000,33.08,1,0.984808,0.173648,80.0,0,2020-01-01 03:53:00,0.001,0.0,68.00,59.00,72.97,5.75,0.0,5.0,3600.0,1017.5,0.0,-0.866025,-0.500000,240.0,0.0,ORD,Early


# Date and Time columns

In [904]:
def cyclical_date_format(df, column, max_value):
    # Create cyclical values for month, day, day_of_week, and/or day_of_year
    df[column + ' (sin)'] = np.sin(2 * np.pi * df[column] / max_value)
    df[column + ' (cos)'] = np.cos(2 * np.pi * df[column] / max_value)
    return df

In [905]:
# Year, Month, Day, Day of Week
delays_df['Year'] = pd.to_datetime(delays_df['Date']).map(lambda x: x.year)
delays_df['Month'] = pd.to_datetime(delays_df['Date']).map(lambda x: x.month)
delays_df['Day'] = pd.to_datetime(delays_df['Date']).map(lambda x: x.day)
delays_df['Day of Week'] = pd.to_datetime(delays_df['Date']).map(lambda x: x.day_of_week)

delays_df['Scheduled Departure Hour'] = pd.to_datetime(delays_df['Scheduled Departure Time'], format='%H:%M').map(lambda x: x.hour)
delays_df['Scheduled Departure Total Minutes'] = pd.to_datetime(delays_df['Scheduled Departure Time'], format='%H:%M').map(lambda x: (x.hour * 60) + x.minute)
delays_df['Certificate Last Activity Date'] = pd.to_datetime(delays_df['Certificate Last Activity Date'])
delays_df['Certificate Issue Date'] = pd.to_datetime(delays_df['Certificate Issue Date'])
delays_df['Air Worthiness Date'] = pd.to_datetime(delays_df['Air Worthiness Date'])
delays_df['Certificate Expiration Date'] = pd.to_datetime(delays_df['Certificate Expiration Date'])

delays_df = cyclical_date_format(delays_df, 'Month', 12)
delays_df = cyclical_date_format(delays_df, 'Day', 31)
delays_df = cyclical_date_format(delays_df, 'Day of Week', 6)
delays_df = cyclical_date_format(delays_df, 'Scheduled Departure Total Minutes', 1440)

delays_df = delays_df.drop(['DateTime', 'Scheduled Arrival DateTime', 'Destination Weather DateTime'], axis=1)
delays_df.head()

,Carrier Code,Date,Flight Number,Tail Number,Destination Airport,Scheduled Departure Time,Actual Departure Time,Scheduled Elapsed Time,Actual Elapsed Time,Departure Delay,Manufacturer Code,Engine Manufacturer Code,Year of Manufacture,Certificate Last Activity Date,Certificate Issue Date,Type of Engine,Air Worthiness Date,Certificate Expiration Date,Manufacturer,Model,Builder Type Certificated,Number of Seats,Precipitation Accumulation One Hour,Air Temperature,Relative Humidity,Wind Speed,Sea Level Pressure,Visibility,Wind Gust,Precipitation Accumulation Six Hours,Ceiling,Dew Point Temperature,Ceiling Missing,Wind Direction (sin),Wind Direction (cos),Wind Direction Interpolation,Sea Level Pressure Missing,Destination Precipication Accumulation One Hour,Destination Precipitation Six Hours,Destination Air Temperature,Destination Dew Point Temperature,Destination Relative Humidity,Destination Wind Speed,Destination Wind Gust,Destination Visibility,Destination Ceiling,Destination Sea Level Pressure,Destination Ceiling Missing,Destination Wind Direction (sin),Destination Wind Direction (cos),Destination Wind Direction Interpolation,Destination Sea Level Pressure Missing,Airport,Delay Bin,Year,Month,Day,Day of Week,Scheduled Departure Hour,Scheduled Departure Total Minutes,Month (sin),Month (cos),Day (sin),Day (cos),Day of Week (sin),Day of Week (cos),Scheduled Departure Total Minutes (sin),Scheduled Departure Total Minutes (cos)
0,F9,01/01/2020,402,N323FR,DEN,00:59,00:46,147,145,-13,3940325,13110,2017,2023-04-07,2017-12-04,Turbo-Fan,2017-12-05,2027-12-31,AIRBUS,A320-251N,True,190.0,0.0,59.00,37.47,6.91,1017.2,10.0,0.0,0.0,35000,33.08,1,0.342020,0.939693,20.0,0,0.000,0.0,69.08,57.92,64.32,5.75,0.0,10.0,4700.0,1017.7,0.0,-0.766044,-0.642788,230.0,0.0,DEN,Early,2020,1,1,2,0,59,0.5,0.866025,0.201299,0.97953,0.866025,-0.5,0.254602,0.967046
1,UA,01/01/2020,1990,N847UA,IAH,00:57,00:50,183,182,-7,3930317,34601,2001,2023-06-16,2018-10-30,Turbo-Jet,2002-01-15,2028-10-31,AIRBUS,A319-131,True,179.0,0.0,59.00,37.47,6.91,1017.2,10.0,0.0,0.0,35000,33.08,1,0.342020,0.939693,20.0,0,0.001,0.0,68.00,59.00,72.97,5.75,0.0,5.0,3600.0,1017.5,0.0,-0.866025,-0.500000,240.0,0.0,IAH,Early,2020,1,1,2,0,57,0.5,0.866025,0.201299,0.97953,0.866025,-0.5,0.246153,0.969231
2,DL,01/01/2020,1969,N543US,MSP,00:40,00:29,209,219,-11,1384970,52117,1996,2023-08-04,2016-05-03,Turbo-Fan,1996-05-15,2029-05-31,BOEING,757-251,True,178.0,0.0,57.02,40.23,5.75,1017.5,10.0,0.0,0.0,35000,33.08,1,0.984808,0.173648,80.0,0,0.001,0.0,68.00,59.00,72.97,5.75,0.0,5.0,3600.0,1017.5,0.0,-0.866025,-0.500000,240.0,0.0,MSP,Early,2020,1,1,2,0,40,0.5,0.866025,0.201299,0.97953,0.866025,-0.5,0.173648,0.984808
3,AA,01/01/2020,1876,N996AN,DFW,01:30,01:48,179,174,18,3940032,34611,2016,2023-09-22,2016-12-20,Turbo-Fan,2016-12-20,2029-12-31,AIRBUS,A321-231,True,379.0,0.0,59.00,37.47,6.91,1017.2,10.0,0.0,0.0,35000,33.08,1,0.342020,0.939693,20.0,0,0.001,0.0,68.00,59.00,72.97,5.75,0.0,5.0,3600.0,1017.5,0.0,-0.866025,-0.500000,240.0,0.0,DFW,11-30 min,2020,1,1,2,1,90,0.5,0.866025,0.201299,0.97953,0.866025,-0.5,0.382683,0.923880
4,UA,01/01/2020,910,N38479,ORD,00:48,00:40,234,238,-8,138488H,13100,2016,2023-09-22,2016-12-23,Turbo-Fan,2016-12-22,2029-12-31,BOEING,737-900ER,True,222.0,0.0,57.02,40.23,5.75,1017.5,10.0,0.0,0.0,35000,33.08,1,0.984808,0.173648,80.0,0,0.001,0.0,68.00,59.00,72.97,5.75,0.0,5.0,3600.0,1017.5,0.0,-0.866025,-0.500000,240.0,0.0,ORD,Early,2020,1,1,2,0,48,0.5,0.866025,0.201299,0.97953,0.866025,-0.5,0.207912,0.978148


# Determine aircraft age

In [906]:
# aircraft_df.to_csv('../Output/Bug_Hunting/aircraft_data.csv', index=False)

In [907]:
delays_df['Year of Manufacture'] = delays_df['Year of Manufacture'].replace('', np.nan).astype('Int64')
delays_df[delays_df['Tail Number'] == 'N2142J']['Year of Manufacture']

200391    <NA>
200926    <NA>
201547    <NA>
202032    <NA>
202702    <NA>
          ... 
892527    <NA>
892946    <NA>
893333    <NA>
893888    <NA>
894272    <NA>
Name: Year of Manufacture, Length: 856, dtype: Int64

In [908]:
# write_csv(delays_df[delays_df['Year of Manufacture'] == ''], '../Output/Bug_Hunting/', 'aircraft_age_unknown', zip=False)

In [909]:
# write_csv(delays_df[delays_df['Year of Manufacture'].isna()].groupby('Tail Number').count().rename({'Date': 'Count'}, axis=1).sort_values('Count')['Count'].reset_index(),
#           '../Output/Bug_Hunting/', 'age_unknown_tail_numbers', zip=False)

In [910]:
delays_df['Aircraft Age'] = delays_df['Year'].astype('Int64') - delays_df['Year of Manufacture'].astype('Int64')
delays_df['Aircraft Age'].value_counts()

Aircraft Age
3     67958
1     63379
2     63310
4     60979
5     60804
6     60060
7     48069
8     40634
9     32349
0     26926
10    26643
20    23554
21    22897
22    21509
19    21220
18    19719
17    18585
23    17855
16    17266
11    16876
15    15851
14    14928
24    13330
13    13254
12    12820
25    11901
26     8245
27     4990
28     3731
29     3140
32     2553
30     2372
31     2012
33     1836
34      822
42        5
43        1
Name: count, dtype: Int64

In [911]:
# Fill Aircraft Age NaN with -9999 and create a Aircraft Age Missing column.
delays_df['Aircraft Age'] = delays_df['Aircraft Age'].fillna(-9999)
delays_df['Aircraft Age Missing'] = delays_df['Aircraft Age'] == -9999

# Check for any other remaining NA values

In [912]:
delays_df[delays_df.isna().any(axis=1)]

,Carrier Code,Date,Flight Number,Tail Number,Destination Airport,Scheduled Departure Time,Actual Departure Time,Scheduled Elapsed Time,Actual Elapsed Time,Departure Delay,Manufacturer Code,Engine Manufacturer Code,Year of Manufacture,Certificate Last Activity Date,Certificate Issue Date,Type of Engine,Air Worthiness Date,Certificate Expiration Date,Manufacturer,Model,Builder Type Certificated,Number of Seats,Precipitation Accumulation One Hour,Air Temperature,Relative Humidity,Wind Speed,Sea Level Pressure,Visibility,Wind Gust,Precipitation Accumulation Six Hours,Ceiling,Dew Point Temperature,Ceiling Missing,Wind Direction (sin),Wind Direction (cos),Wind Direction Interpolation,Sea Level Pressure Missing,Destination Precipication Accumulation One Hour,Destination Precipitation Six Hours,Destination Air Temperature,Destination Dew Point Temperature,Destination Relative Humidity,Destination Wind Speed,Destination Wind Gust,Destination Visibility,Destination Ceiling,Destination Sea Level Pressure,Destination Ceiling Missing,Destination Wind Direction (sin),Destination Wind Direction (cos),Destination Wind Direction Interpolation,Destination Sea Level Pressure Missing,Airport,Delay Bin,Year,Month,Day,Day of Week,Scheduled Departure Hour,Scheduled Departure Total Minutes,Month (sin),Month (cos),Day (sin),Day (cos),Day of Week (sin),Day of Week (cos),Scheduled Departure Total Minutes (sin),Scheduled Departure Total Minutes (cos),Aircraft Age,Aircraft Age Missing
13,DL,01/01/2020,2377,N383DN,SLC,06:00,05:59,120,128,-1,13844CF,13802,1999,2023-06-03,1999-10-20,Turbo-Fan,1999-10-18,2028-08-31,BOEING,737-832,True,189.0,0.0,51.08,49.97,4.61,1017.7,10.0,0.0,0.0,35000,33.08,1,0.939693,3.420201e-01,70.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Early,2020,1,1,2,6,360,5.000000e-01,0.866025,2.012985e-01,0.97953,0.866025,-0.5,1.000000,6.123234e-17,21,False
19,WN,01/01/2020,4207,N799SW,TUS,07:10,07:04,85,77,-6,1384418,,<NA>,2023-10-07,2017-02-15,Turbo-Fan,NaT,2030-02-28,BOEING,737-7Q8,True,149.0,0.0,53.06,42.58,0.00,1017.8,10.0,0.0,0.0,35000,30.92,1,0.000000,1.000000e+00,0.0,0,0.00,0.001,71.06,60.98,68.81,3.44,0.0,10.0,4300.0,1018.50,0.0,-0.642788,-0.766044,220.0,0.0,TUS,Early,2020,1,1,2,7,430,5.000000e-01,0.866025,2.012985e-01,0.97953,0.866025,-0.5,0.953717,-3.007058e-01,-9999,True
33,DL,01/01/2020,1404,N357NW,SLC,07:35,07:35,109,120,0,3930322,13802,1998,2023-08-11,2009-12-31,Turbo-Fan,1998-07-02,2029-06-30,AIRBUS,A320-212,True,182.0,0.0,53.06,42.58,0.00,1017.8,10.0,0.0,0.0,35000,30.92,1,0.000000,1.000000e+00,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,On Time,2020,1,1,2,7,455,5.000000e-01,0.866025,2.012985e-01,0.97953,0.866025,-0.5,0.915311,-4.027467e-01,22,False
38,OO,01/01/2020,5553,N204SY,SLC,07:44,08:09,123,140,25,3260415,30061,2016,2023-09-22,2016-12-21,Turbo-Fan,2016-12-21,2029-12-31,EMBRAER,ERJ 170-200 LR,True,88.0,0.0,53.06,42.58,0.00,1017.8,10.0,0.0,0.0,35000,30.92,1,0.000000,1.000000e+00,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11-30 min,2020,1,1,2,7,464,5.000000e-01,0.866025,2.012985e-01,0.97953,0.866025,-0.5,0.898794,-4.383711e-01,4,False
104,DL,01/01/2020,2658,N3740C,SLC,10:03,10:00,114,110,-3,13844CF,13802,2000,2023-08-04,2000-12-21,Turbo-Fan,2000-12-19,2029-05-31,BOEING,737-832,True,189.0,0.0,62.06,42.53,5.75,1018.5,10.0,0.0,0.0,25000,39.02,0,1.000000,6.123234e-17,90.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Early,2020,1,1,2,10,603,5.000000e-01,0.866025,2.012985e-01,0.97953,0.866025,-0.5,0.488621,-8.724960e-01,20,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894539,DL,12/31/2024,2311,N387DA,SLC,17:00,16:56,112,95,-4,13844CF,13802,2000,2023-08-11,2000-01-18,Turbo-Fan,2000-01-13,2029-06-3

In [913]:
len(delays_df)

854617

# Reorder columns

In [914]:
pre_columns = delays_df.columns

In [915]:
print(f'Number of columns (before):  {len(delays_df.columns)}')
delays_df = delays_df[['Delay Bin', 'Departure Delay', 'Airport', 'Date', 'Year', 'Month', 'Day', 'Day of Week',
                       'Month (sin)', 'Month (cos)', 'Day (sin)', 'Day (cos)', 'Day of Week (sin)', 'Day of Week (cos)',
                       'Scheduled Departure Total Minutes (sin)', 'Scheduled Departure Total Minutes (cos)', 'Scheduled Departure Total Minutes',
                       'Scheduled Departure Hour', 'Scheduled Departure Time', 'Actual Departure Time', 'Scheduled Elapsed Time', 'Actual Elapsed Time',
                       'Carrier Code', 'Flight Number', 'Tail Number', 'Destination Airport',
                       'Manufacturer Code', 'Manufacturer', 'Model', 'Engine Manufacturer Code', 'Year of Manufacture', 'Aircraft Age', 'Aircraft Age Missing',
                       'Type of Engine', 'Number of Seats',
                       'Certificate Last Activity Date', 'Certificate Issue Date', 'Air Worthiness Date', 'Certificate Expiration Date', 'Builder Type Certificated',
                       'Precipitation Accumulation One Hour', 'Precipitation Accumulation Six Hours', 'Air Temperature', 'Dew Point Temperature',
                       'Relative Humidity', 'Wind Speed', 'Wind Direction Interpolation', 
                       'Wind Direction (sin)', 'Wind Direction (cos)',
                       'Wind Gust', 'Visibility', 'Ceiling', 'Ceiling Missing',
                       'Sea Level Pressure', 'Sea Level Pressure Missing',
                       'Destination Precipication Accumulation One Hour', 'Destination Precipitation Six Hours', 'Destination Air Temperature',
                       'Destination Dew Point Temperature', 'Destination Relative Humidity', 'Destination Wind Speed',
                       'Destination Wind Direction Interpolation', 'Destination Wind Direction (sin)', 'Destination Wind Direction (cos)',
                       'Destination Wind Gust', 'Destination Visibility', 'Destination Ceiling',  'Destination Ceiling Missing', 
                       'Destination Sea Level Pressure', 'Destination Sea Level Pressure Missing']]

# Ensure we have the same number of columns, just reordered
print(f'Number of columns (after):  {len(delays_df.columns)}')

Number of columns (before):  70
Number of columns (after):  70


In [916]:
post_columns = delays_df.columns

In [917]:
# List any columns not in both the pre and post lists
for column in pre_columns:
    if column not in post_columns:
        print(column)

In [918]:
# Output the full dataset
write_csv(delays_df, '../../../Preprocessing/Output/Delays/', 'flight_delays_dest', zip=True)

Zipping "flight_delays_dest.csv" into "flight_delays_dest.zip"
Writing to "../../../Preprocessing/Output/Delays/flight_delays_dest.zip"


In [919]:
delays_df.columns

Index(['Delay Bin', 'Departure Delay', 'Airport', 'Date', 'Year', 'Month',
       'Day', 'Day of Week', 'Month (sin)', 'Month (cos)', 'Day (sin)',
       'Day (cos)', 'Day of Week (sin)', 'Day of Week (cos)',
       'Scheduled Departure Total Minutes (sin)',
       'Scheduled Departure Total Minutes (cos)',
       'Scheduled Departure Total Minutes', 'Scheduled Departure Hour',
       'Scheduled Departure Time', 'Actual Departure Time',
       'Scheduled Elapsed Time', 'Actual Elapsed Time', 'Carrier Code',
       'Flight Number', 'Tail Number', 'Destination Airport',
       'Manufacturer Code', 'Manufacturer', 'Model',
       'Engine Manufacturer Code', 'Year of Manufacture', 'Aircraft Age',
       'Aircraft Age Missing', 'Type of Engine', 'Number of Seats',
       'Certificate Last Activity Date', 'Certificate Issue Date',
       'Air Worthiness Date', 'Certificate Expiration Date',
       'Builder Type Certificated', 'Precipitation Accumulation One Hour',
       'Precipitation Accum

In [920]:
# For modeling:
# Drop Date, Year, Month, Day, Day of Week, Scheduled Departure Time in favor of sin/cos cyclical variants
# Drop Year because it can't be used for future predictions
# Drop unique identifiers: Flight Number, Tail Number
# Drop Scheduled Departure Hour as it is redundant with Scheduled Departure Time and will be used for other features
# Drop aircraft dates (Certificate, Air Worthiness) since we're not considering dates in the model. Also, Air Worthiness seems synonymous with Year of Manufacture
# Drop redundant columns: Manufacturer Code, Engine Manufacturer Code
# Drop Year of Manufacture in favor of Aircraft Age & Aircraft Age Missing (for NaN values)
# Drop actual times because they occur at/after departure and can't be used for prediction

# NOTE: Only use 1 of Delay Bin or Departure Delay for the model target. Drop the other.

model_df = delays_df.drop(['Date', 'Year', 'Month', 'Day', 'Day of Week', 'Airport',
                           'Flight Number', 'Tail Number', 'Departure Delay',
                           'Scheduled Departure Hour', 'Scheduled Departure Time', 'Scheduled Departure Total Minutes',
                           'Certificate Last Activity Date', 'Certificate Issue Date', 'Air Worthiness Date', 'Certificate Expiration Date',
                           'Manufacturer Code', 'Engine Manufacturer Code', 'Year of Manufacture',
                           'Actual Departure Time', 'Actual Elapsed Time'], axis=1)
print(model_df.columns)
print(f'Number of columns:  {len(model_df.columns)}')

Index(['Delay Bin', 'Month (sin)', 'Month (cos)', 'Day (sin)', 'Day (cos)',
       'Day of Week (sin)', 'Day of Week (cos)',
       'Scheduled Departure Total Minutes (sin)',
       'Scheduled Departure Total Minutes (cos)', 'Scheduled Elapsed Time',
       'Carrier Code', 'Destination Airport', 'Manufacturer', 'Model',
       'Aircraft Age', 'Aircraft Age Missing', 'Type of Engine',
       'Number of Seats', 'Builder Type Certificated',
       'Precipitation Accumulation One Hour',
       'Precipitation Accumulation Six Hours', 'Air Temperature',
       'Dew Point Temperature', 'Relative Humidity', 'Wind Speed',
       'Wind Direction Interpolation', 'Wind Direction (sin)',
       'Wind Direction (cos)', 'Wind Gust', 'Visibility', 'Ceiling',
       'Ceiling Missing', 'Sea Level Pressure', 'Sea Level Pressure Missing',
       'Destination Precipication Accumulation One Hour',
       'Destination Precipitation Six Hours', 'Destination Air Temperature',
       'Destination Dew Point Temp

# Check for any remaining null values in the model dataframe
## These will choke machine learning algorithms

In [921]:
model_df.isna().any()

Delay Bin                                          False
Month (sin)                                        False
Month (cos)                                        False
Day (sin)                                          False
Day (cos)                                          False
Day of Week (sin)                                  False
Day of Week (cos)                                  False
Scheduled Departure Total Minutes (sin)            False
Scheduled Departure Total Minutes (cos)            False
Scheduled Elapsed Time                             False
Carrier Code                                       False
Destination Airport                                False
Manufacturer                                       False
Model                                              False
Aircraft Age                                       False
Aircraft Age Missing                               False
Type of Engine                                     False
Number of Seats                

In [922]:
model_df[model_df.isna().any(axis=1)]

,Delay Bin,Month (sin),Month (cos),Day (sin),Day (cos),Day of Week (sin),Day of Week (cos),Scheduled Departure Total Minutes (sin),Scheduled Departure Total Minutes (cos),Scheduled Elapsed Time,Carrier Code,Destination Airport,Manufacturer,Model,Aircraft Age,Aircraft Age Missing,Type of Engine,Number of Seats,Builder Type Certificated,Precipitation Accumulation One Hour,Precipitation Accumulation Six Hours,Air Temperature,Dew Point Temperature,Relative Humidity,Wind Speed,Wind Direction Interpolation,Wind Direction (sin),Wind Direction (cos),Wind Gust,Visibility,Ceiling,Ceiling Missing,Sea Level Pressure,Sea Level Pressure Missing,Destination Precipication Accumulation One Hour,Destination Precipitation Six Hours,Destination Air Temperature,Destination Dew Point Temperature,Destination Relative Humidity,Destination Wind Speed,Destination Wind Direction Interpolation,Destination Wind Direction (sin),Destination Wind Direction (cos),Destination Wind Gust,Destination Visibility,Destination Ceiling,Destination Ceiling Missing,Destination Sea Level Pressure,Destination Sea Level Pressure Missing
13,Early,5.000000e-01,0.866025,2.012985e-01,0.97953,0.866025,-0.5,1.000000,6.123234e-17,120,DL,SLC,BOEING,737-832,21,False,Turbo-Fan,189.0,True,0.0,0.0,51.08,33.08,49.97,4.61,70.0,0.939693,3.420201e-01,0.0,10.0,35000,1,1017.7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,On Time,5.000000e-01,0.866025,2.012985e-01,0.97953,0.866025,-0.5,0.915311,-4.027467e-01,109,DL,SLC,AIRBUS,A320-212,22,False,Turbo-Fan,182.0,True,0.0,0.0,53.06,30.92,42.58,0.00,0.0,0.000000,1.000000e+00,0.0,10.0,35000,1,1017.8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,11-30 min,5.000000e-01,0.866025,2.012985e-01,0.97953,0.866025,-0.5,0.898794,-4.383711e-01,123,OO,SLC,EMBRAER,ERJ 170-200 LR,4,False,Turbo-Fan,88.0,True,0.0,0.0,53.06,30.92,42.58,0.00,0.0,0.000000,1.000000e+00,0.0,10.0,35000,1,1017.8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,Early,5.000000e-01,0.866025,2.012985e-01,0.97953,0.866025,-0.5,0.488621,-8.724960e-01,114,DL,SLC,BOEING,737-832,20,False,Turbo-Fan,189.0,True,0.0,0.0,62.06,39.02,42.53,5.75,90.0,1.000000,6.123234e-17,0.0,10.0,25000,0,1018.5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,Early,5.000000e-01,0.866025,2.012985e-01,0.97953,0.866025,-0.5,0.414693,-9.099613e-01,130,OO,SLC,EMBRAER,ERJ 170-200 LR,4,False,Turbo-Fan,88.0,True,0.0,0.0,62.06,39.02,42.53,5.75,90.0,1.000000,6.123234e-17,0.0,10.0,25000,0,1018.5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894464,Early,-2.449294e-16,1.000000,-2.449294e-16,1.00000,0.866025,0.5,-0.608761,-7.933533e-01,119,F9,SLC,AIRBUS,A321-271NX,0,False,Turbo-Fan,246.0,True,0.0,0.0,60.08,48.02,64.38,6.91,280.0,-0.984808,1.736482e-01,0.0,5.0,14000,0,1016.8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
894483,Early,-2.449294e-16,1.000000,-2.449294e-16,1.00000,0.866025,0.5,-0.731354,-6.819984e-01,124,OO,SLC,EMBRAER,ERJ 170-200 LL,5,False,Turbo-Fan,88.0,True,0.0,0.0,60.98,48.02,62.36,11.51,270.0,-1.000000,-1.836970e-16,0.0,4.0,14000,0,1017.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
894539,Early,-2.449294e-16,1.000000,-2.449294e-16,1.00000,0.866025,0.5,-0.965926,-2.588190e-01,112,DL,SLC,BOEING,737-832,24,False,Turbo-Fan,189.0,True,0.0,0.0,53.96,44.06,69.10,11.51,270.0,-1.000000,-1.836970e-16,0.0,3.0,1000,0,1017.2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
894594,Early,-2.449294e-16,1.000000,-2.449294e-16,1.00000,0.866025,0.5,-0.946930,3.214395e-01,112,DL,SLC,BOEING,737-900ER,6,False,Turbo-Fan,222.0,True,0.0,0.0,55.04,44.96,68.75,5.75,240.0,-0.866025,-5.000000e-01,0.0,3.0,1000,0,1018.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [923]:
len(model_df)

854617

In [924]:
model_df[model_df.isna().any(axis=1)]['Destination Airport'].value_counts()

Destination Airport
SLC    22910
SUN     1051
FCA      750
Name: count, dtype: int64

In [925]:
# Drop the remaining rows with null values for machine learning compatibility
model_df = model_df.dropna()
display(model_df[model_df.isna().any(axis=1)])
print(len(model_df))


,Delay Bin,Month (sin),Month (cos),Day (sin),Day (cos),Day of Week (sin),Day of Week (cos),Scheduled Departure Total Minutes (sin),Scheduled Departure Total Minutes (cos),Scheduled Elapsed Time,Carrier Code,Destination Airport,Manufacturer,Model,Aircraft Age,Aircraft Age Missing,Type of Engine,Number of Seats,Builder Type Certificated,Precipitation Accumulation One Hour,Precipitation Accumulation Six Hours,Air Temperature,Dew Point Temperature,Relative Humidity,Wind Speed,Wind Direction Interpolation,Wind Direction (sin),Wind Direction (cos),Wind Gust,Visibility,Ceiling,Ceiling Missing,Sea Level Pressure,Sea Level Pressure Missing,Destination Precipication Accumulation One Hour,Destination Precipitation Six Hours,Destination Air Temperature,Destination Dew Point Temperature,Destination Relative Humidity,Destination Wind Speed,Destination Wind Direction Interpolation,Destination Wind Direction (sin),Destination Wind Direction (cos),Destination Wind Gust,Destination Visibility,Destination Ceiling,Destination Ceiling Missing,Destination Sea Level Pressure,Destination Sea Level Pressure Missing


829906


In [926]:
# Output the modeling dataset
write_csv(model_df, '../../../Preprocessing/Output/Delays/', 'modeling_data_dest', zip=True)

Zipping "modeling_data_dest.csv" into "modeling_data_dest.zip"
Writing to "../../../Preprocessing/Output/Delays/modeling_data_dest.zip"
